# Create and Run chat flows using Promptflow Python SDK 

### Overview
Prompt flow is a suite of development tools designed to streamline the end-to-end development cycle of LLM-based AI applications, from ideation, prototyping, testing, evaluation to production deployment and monitoring. It makes prompt engineering much easier and enables you to build LLM apps with production quality. 

In this hands-on, you will be able to:
Create flows that link fine-tuned phi3.5 endpoint(Python code) and gpt model in a executable workflow.
Debug and iterate your flows, especially tracing interaction with LLMs with ease.


#### 1. Set up Promptflow client with Credential and configuration
#### 2. Create a new chat flow by providing the flow name and description.
#### 3. Run Basic Promptflow with questions to compare models
#### 4. Run Context Added Promptflow with the outdoor questions 
#### 5. Use serverless endpoint to run the Promptflow with context

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [1]:
import json
import os
import time

# Import required libraries
from promptflow.azure import PFClient
from promptflow.entities import Run
# Import required libraries
from azure.identity import DefaultAzureCredential, EnvironmentCredential, InteractiveBrowserCredential
from dotenv import load_dotenv
from azure.core.exceptions import HttpResponseError
load_dotenv()
with open('./config.json', 'r') as f:
    config = json.load(f)
    
print(config["subscription_id"])
print(config["resource_group"])
print(config["workspace_name"]) # Azure AI Studio project name which is not the same as the Azure ML workspace name

3d4d3dd0-79d4-40cf-a94e-b4154812c6ca
slm-innovator-rg
slm-pjt1


In [2]:
from tqdm import tqdm

# Monitor the status of the run_result
def monitor_status(pf_azure_client:PFClient, run_result:Run):
    with tqdm(total=3, desc="Running Status", unit="step") as pbar:
        status = pf_azure_client.runs.get(run_result).status
        if status == "Preparing":
            pbar.update(1)
        while status != "Completed" and status != "Failed":
            if status == "Running" and pbar.n < 2:
                pbar.update(1)
            print(f"Current Status: {status}")
            time.sleep(10)
            status = pf_azure_client.runs.get(run_result).status
        pbar.update(1)
        print("Promptflow Running Completed")

## 1. Set up Promptflow client with Credential and configuration
- Create a promptflow client with the credential and configuration. You need to set the `config.json` file with subscription_id, resource_group and workspace_name

In [5]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()
# if you cannot use DefaultAzureCredential and InteractiveBrowserCredential you need to set up the Managed identity in your .env file

pf_azure_client = PFClient.from_config(credential=credential, path="./config.json")

# pf_azure_client = PFClient(credential=credential, 
#                            subscription_id="your subscription id", 
#                            resource_group_name="your resource group name", 
#                            workspace_name="your workspace name")            

try:
    workspace = pf_azure_client.ml_client.workspaces.get(name=config["workspace_name"])
    print(f"Connected to Azure AI Studio Workspace: {workspace.name}")
    print(f"Workspace Location: {workspace.location}")
    print(f"Workspace ID: {workspace.id}")
except HttpResponseError as e:
    print(f"Failed to connect to Azure ML Workspace: {e}")


Found the config file in: config.json


Connected to Azure AI Studio Workspace: slm-pjt1
Workspace Location: eastus
Workspace ID: /subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourceGroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


## Check the exist connections
- currently we only support create connection in Azure AI, ML Studio UI. Check the exiting connections in the workspace.

In [6]:
from promptflow.client import PFClient
from promptflow.entities import AzureOpenAIConnection, CustomConnection

# Get a pf client to get connections
# currently we only support create connection in Azure AI, ML Studio UI
pf = PFClient()
try:
    pf.connections.list()
    for connection in pf.connections.list():
        print(connection)
except (ValueError, TypeError) as e:
    print(e)

## 2. Create a new chat flow by providing the flow name and description.
- Create a new chat flow by providing the flow name and description. You can view and clone the flow on Azure AI studio UI. 

In [ ]:
#Storage Blob Data Contributor, Storage File Data Privileged Contributor role을 managed Identity / service principal / InteractiveBrowserCredential 에게 부여하면 아래 코드를 실행할 수 있습니다.
pf_azure_client.flows.create_or_update(flow="chat/", type="chat", display_name="comparison flow created from python sdk", description="fine-tuned model comparison flow")

## 2. Run Basic Promptflow with questions to compare models
- Run the Promptflow with the simple questions such as "What is the capital of France?" and compare the results of the models

In [8]:
flow_path = "./chat"
data_path = "./data/questions_basic.jsonl"

column_mapping = {
    "question": "${data.question}"
}

run_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path,
    column_mapping=column_mapping,
    display_name="chat_with_data",
    tags={"chat_with_jsonl": "", "1st_round": ""},
)


Uploading chat (0.04 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36807/36807 [00:00<00:00, 527927.57it/s]


Portal url: https://ai.azure.com/projectflows/trace/run/chat_variant_0_20241025_141356_267978/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [9]:
monitor_status(pf_azure_client, run_result)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [01:13<02:27, 73.76s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [01:24<00:36, 36.87s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:03<00:00, 81.04s/step]

Promptflow Running Completed


In [10]:
detail = pf_azure_client.get_details(run_result)

detail

,inputs.question,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,
0,what is the center of Seoul?,0,The center of Seoul is often considered to be...,The center of Seoul is often considered to be ...
1,What is the capital of France?,1,The capital of France is Paris. Paris is the ...,The capital of France is Paris.
2,Tell me about the brief history of Microsoft,2,Microsoft started in 1975 by Bill Gates and P...,Microsoft was founded by Bill Gates and Paul A...
3,Who wrote Romeo and Juliet?,3,"William Shakespeare wrote ""Romeo and Juliet.""...","William Shakespeare wrote ""Romeo and Juliet."""
4,What does HTML stand for?,4,HTML stands for HyperText Markup Language. It...,HTML stands for HyperText Markup Language.
5,Who painted the Mona Lisa?,5,The Mona Lisa was painted by Leonardo da Vinc...,Leonardo da Vinci.
6,What is the largest planet in our solar system?,6,Jupiter is the largest planet in our solar sy...,Jupiter is the largest planet in our solar sys...
7,Who won the Nobel Prize in Literature in 2024?,7,None,"I'm sorry, but I don't have information on Nob..."


## 3. Run Context Added Promptflow with the outdoor questions 
- Run the Promptflow using the context data and ask the outdoor product related questions to compare the results of the models

In [11]:
flow_path = "./chat-context"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_result_with_context = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_context_data",
    tags={"chat_with_context_jsonl": "", "1st_round": ""},
)


[2024-10-25 14:18:17 +0000][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2024-10-25 14:18:17 +0000][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/chat_context_variant_0_20241025_141815_867099/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [12]:
monitor_status(pf_azure_client, run_result_with_context)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:52<01:45, 52.99s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [01:03<00:28, 28.00s/step]

Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [04:03<00:00, 81.07s/step]

Promptflow Running Completed


In [13]:
detail = pf_azure_client.get_details(run_result_with_context)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,None,TrailMaster X4 텐트는 4인용으로 설계된 실용적인 캠핑 텐트입니다. 내구...
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,"저는 AI이므로 실질적으로 기기나 장비를 가지고 있지 않지만, 기타 정보나 제안을...","클라이밍 장비 찾고 계신가요? 😊 \n암벽화, 카라비너, 하네스 등 다양한 장비가..."
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,트레일마스터 X4 텐트는 강성 폴리에스터로 제작된 4명 용도로 물에 강하게 만들어...,저희 텐트 종류에 대해 설명드릴게요! 🏕️\n\n1. **돔 텐트**: 설치가 간편...
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,None,"죄송하지만, TrekReady 하이킹 부츠의 가격은 제가 직접 제공할 수 없습니다...."
4,can you tell me BaseCamp Folding Table?,TrailMaster X4 Tent is a durable polyester ten...,4,None,BaseCamp 접이식 테이블에 대해 알려드릴게요! 이 테이블은 가볍고 휴대하기 쉬...


## 4. Use serverless endpoint to run the Promptflow with context
- Create a serverless endpoint to run the Promptflow with the context. You can use the endpoint to run the flow with the context.

### deploy your serverless endpoint

- go to the Azure AI studio > Model catalog > search phi-3.5 > deply Phi-3.5-mini-instruct as your serverless endpint 
<br>
![serverless endpoint](./images/deploy_serverless_endpoint.jpg)
<br>
<br>
- once the deployment is done, go to Deployments and you can see the endpoint deployed in the endpoint section. Click to check the details and copy key and phi35-mini-instruct: Chat Completion endpoint url
![copy connection](./images/copy_connection.jpg)
<br>
<br>
- go to Settings in Azure AI studio > Connections > create a new connection naming phi35-serverless with the copied key and endpoint url
![create new serverless connection](./images/create_new_serverless_connection.jpg)





In [14]:
flow_path = "./chat-serverless"
data_path = "./data/questions_outdoor.jsonl"

# get the context from context.json file as str and map it to the column_mapping
with open('./data/context_simple.json', 'r') as file:
    context = json.load(file)

column_mapping = {
    "question": "${data.question}",
    "context": context.get("context")    
}

run_serverless_result = pf_azure_client.run(
    flow=flow_path,
    type="chat",
    data=data_path, 
    column_mapping=column_mapping,
    display_name="chat_serverless_context_data",
    tags={"chat_serverless_context_jsonl": "", "1st_round": ""},
)


[2024-10-25 14:22:27 +0000][promptflow][WARNING] - You're using compute session, if it's first time you're using it, it may take a while to build session and you may see 'NotStarted' status for a while. 
[2024-10-25 14:22:27 +0000][promptflow][WARNING] - The trace Cosmos DB for current workspace/project is not ready yet, your traces might not be logged and stored properly.
To enable it, please run `pf config set trace.destination=azureml://subscriptions/<subscription-id>/resourceGroups/<resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<workspace-or-project-name>`, prompt flow will help to get everything ready.



Portal url: https://ai.azure.com/projectflows/trace/run/chat_serverless_variant_0_20241025_142226_832733/details?wsid=/subscriptions/3d4d3dd0-79d4-40cf-a94e-b4154812c6ca/resourcegroups/slm-innovator-rg/providers/Microsoft.MachineLearningServices/workspaces/slm-pjt1


In [15]:
monitor_status(pf_azure_client, run_serverless_result)

Running Status:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/3 [00:00<?, ?step/s]

Current Status: NotStarted
Current Status: NotStarted
Current Status: NotStarted
Current Status: Preparing


Running Status:  33%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 1/3 [00:42<01:24, 42.15s/step]

Current Status: Running


Running Status:  67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                                                                                                                                                                                                                                                                           | 2/3 [00:52<00:23, 23.52s/step]

Current Status: Running
Current Status: Running


Running Status: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:13<00:00, 24.58s/step]

Promptflow Running Completed


In [16]:
detail = pf_azure_client.get_details(run_serverless_result)

detail

,inputs.question,inputs.context,inputs.line_number,outputs.phi35_answer,outputs.gpt4o_answer
outputs.line_number,,,,,
0,tell me about your TrailMaster X4 Tent,TrailMaster X4 Tent is a durable polyester ten...,0,"TrailMaster X4 Tent은 강력한 폴리에스터로 만들어진 활주로 텐트로,...","TrailMaster X4 텐트는 4인용 캠핑 텐트로, 설치가 쉽고 내구성이 뛰어난..."
1,Do you have any climbing gear?,TrailMaster X4 Tent is a durable polyester ten...,1,"네, 기술 도구를 가지고 있습니다. 언급하신 계곡 산악 장비에 대해 알려드릴게요:...","클라이밍 장비 찾고 계신가요? 😊 헬멧, 하네스, 카라비너 등 다양한 장비가 있어요..."
2,Can you tell me about your selection of tents?,TrailMaster X4 Tent is a durable polyester ten...,2,트레일마스터 X4는 풀 트레프용으로 강력한 고기본으로 구성된 네 명의 사람을 수용...,"저희 텐트 컬렉션은 다양합니다! 🏕️ 다양한 크기와 스타일로 캠핑, 하이킹, 글램핑..."
3,Do you have TrekReady Hiking Boots? How much i...,TrailMaster X4 Tent is a durable polyester ten...,3,트레크리에어 힐크스 야외 복장은 지지와 편안함을 제공합니다. 그러나 구체적인 가격...,TrekReady 하이킹 부츠를 찾으시는군요! 😄 현재 재고 정보는 확인이 안 되지...
4,can you tell me BaseCamp Folding Table?,TrailMaster X4 Tent is a durable polyester ten...,4,"기본캠프 플랫폼은 48x24인치의 가벼운 알루미늄 테이블로, 펼쳐지는 디자인으로 ...",BaseCamp 접이식 테이블은 가벼우면서도 튼튼한 캠핑 테이블이에요. 설치와 보관...
